# Python Lab 03: Krylov Methods
## Francesco Della Santa, Computational Linear Algebra for Large Scale Problems, Politecnico di Torino

In this lesson, we will implement the (Modified) *Arnoldi* (A) method and the *Full Orthogonalization Method* (FOM) method.

In [4]:
# ***** ATTENTION! *****
# If you want that the "%matplotlib widget" works, you need the package ipympl (pip install ipympl)
#
#
# MATPLOTLIB INTERACTIVE VISUALIZATION. REMOVE (OR COMMENT) IF YOU NEED TO PRINT THE NOTEBOOK AS A PDF, SOMETIMES IT DOES NOT WORK WELL...
# %matplotlib widget
#
#

from IPython.display import display  # to display variables in a "nice" way
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from testmatrices import A0, b0, Ab0, v

ModuleNotFoundError: No module named 'numpy'

## Krylov Subspaces

The Krylov methods exploit the *Krylov subspaces* to factorize matrices and solve linear systems. Before introducing the Krylov subspaces, we recall the *minimal polynomials* of a square matrix.

### Minimal Polynomials

Let $A\in\mathbb{R}^{n\times n}$ be a square matrix. Then, the *minimal polynomial of* $A$ is the polynomial $p_A:\mathbb{R}^{n\times n}\rightarrow\mathbb{R}^{n\times n}$ of degree $\kappa$, i.e.
$$p_A(X) = X^{\kappa} + c_{\kappa-1}X^{\kappa-1} + \cdots + c_0 \underbrace{\mathbb{I}_n}_{=X^0}\,,$$ 
where $\kappa\in\mathbb{N}$ is the *lowest degree* such that $p_A(A)=\mathbb{0}_n$.

Now, let $\boldsymbol{v}\in\mathbb{R}^n\setminus\{\boldsymbol{0}\}$ be a non-null vector. Then, the *minimal polynomial of* $\boldsymbol{v}$ *w.r.t.* $A$ is the polynomial $p_{A,\boldsymbol{v}}:\mathbb{R}^{n\times n}\rightarrow\mathbb{R}^{n\times n}$ of degree $\mu$, i.e.
$$p_{A,\boldsymbol{v}}(X) = X^{\mu} + c_{\mu-1}X^{\mu-1} + \cdots + c_0 \mathbb{I}_n\,,$$ 
where $\mu\in\mathbb{N}$ is the *lowest degree* such that $p_{A,\boldsymbol{v}}(A)\boldsymbol{v}=\boldsymbol{0}$. In particular, we define $\mu$ as the *grade of* $\boldsymbol{v}$ *w.r.t.* $A$ and we denote it by $\mathrm{grade}(A,\boldsymbol{v})$.


**Observations:**
1. $A\in\mathbb{R}^{n\times n}$ is invertible *if and only if* $\kappa=n$ and $c_0\neq 0$;

2. Let $A$ be invertible, then
$$A^{-1} = -\frac{1}{c_0}\left(A^{n-1} + c_{n-1}A^{n-2} + \cdots + c_1 \mathbb{I}_n \right) := p^{inv}(A)\,.$$



### Krylov Subspaces


Given a square matrix $A\in\mathbb{R}^{n\times n}$, a non-null vector $\boldsymbol{v}\in\mathbb{R}^n$, and a fixed $m\in\mathbb{N}$, we define *order-*$m$ *Krylov subspace generated by* $A$ *and* $\boldsymbol{v}$ the subspace of $\mathbb{R}^n$:
$$\mathcal{K}_m(A,\boldsymbol{v}):= \langle\boldsymbol{v}, A\boldsymbol{v},\ldots, A^{m-1}\boldsymbol{v}\rangle=\left\lbrace \boldsymbol{x}\in\mathbb{R}^n \ \left| \ \boldsymbol{x}=\sum_{i=1}^m \lambda_{i-1}A^{i-1}\boldsymbol{v} \right . \right\rbrace\,.$$

**Observations:**
1. $A^{-1}\boldsymbol{v} = p^{inv}(A)\boldsymbol{v}\in\mathcal{K}_n(A,\boldsymbol{v})$, for each invertible matrix $A\in\mathbb{R}^{n\times n}$ and each non-null $\boldsymbol{v}\in\mathbb{R}^n$;

1. Let $\mu\in\mathbb{N}$ be $\mu=\mathrm{grade}(A,\boldsymbol{v})$. Then, $\mathrm{dim}\left(\mathcal{K}_m(A,\boldsymbol{v})\right) = \min(m, \mu)$;

1. For each $m\geq \mu=\mathrm{grade}(A,\boldsymbol{v})$, it holds $\mathcal{K}_m(A,\boldsymbol{v})\equiv\mathcal{K}_\mu(A,\boldsymbol{v})$;

1. Let $A\in\mathbb{R}^{n\times n}$ be invertible and let $\boldsymbol{b}\in\mathbb{R}^n$ be a non-null vector. Let $\boldsymbol{x}^*\in\mathbb{R}^n$ be the solution of the linear system $A\boldsymbol{x}=\boldsymbol{b}$. Then:

    1. $\boldsymbol{x}^*=A^{-1}\boldsymbol{b}=p^{inv}(A)\boldsymbol{b}\in\mathcal{K}_n(A, \boldsymbol{b})$;
    
    1. Given an initial guess $\boldsymbol{x}_0\in\mathbb{R}^n$ for the linear system $A\boldsymbol{x}=\boldsymbol{b}$ and the residual $\boldsymbol{r}_0= \boldsymbol{b} - A\boldsymbol{x}_0$, then
    $$\boldsymbol{x}^* = A^{-1}\boldsymbol{b}=A^{-1}(A\boldsymbol{x}_0+\boldsymbol{r}_0) = \underbrace{\boldsymbol{x}_0}_{\in\mathbb{R}^n} + \underbrace{A^{-1}\boldsymbol{r}_0}_{\in\mathcal{K}_n(A,\boldsymbol{r}_0)}\,.$$

1. Let $A\in\mathbb{R}^{n\times n}$ be a square matrix, let $\boldsymbol{v}\in\mathbb{R}^n$ be a non-null vector, and let $m,m_0\in\mathbb{N}$ be such that $m=\mathrm{dim}(\mathcal{K}_{m_0}(A,\boldsymbol{v}))$. Let $V\in\mathbb{R}^{n\times m}$ be the (possibly *rectangular*) matrix where the columns are an orthogonal basis of $\mathcal{K}_{m_0}(A,\boldsymbol{v})$, i.e. $V^T V=\mathbb{I}_m\in\mathbb{R}^{m\times m}$. Then:
    1. $V^T A V = H\in\mathbb{R}^{m\times m}$ is an *upper Hessenberg matrix* (i.e., $h_{ij}=0$ for each $i>j+1$, a sort of "soft" upper triangular matrix);
    
    1. the vectors $\boldsymbol{w}_{j+1}:= A\boldsymbol{v}_j - \sum_{i=1}^j h_{ij}\boldsymbol{v}_i$ have norm $||\boldsymbol{w}_{j+1}||>0$ if $j=1,\ldots,\mu-1$ and norm $||\boldsymbol{w}_{j+1}||=0$ if $j\geq\mu$, where $\mu=\mathrm{grade}(A,\boldsymbol{v})$;
    
    1. $AV = \bar{V}\bar{H}\in\mathbb{R}^{n\times m}$, if $m=m_0<\mu$. Where
    $$ \bar{V} := 
    \begin{bmatrix}
    V & | & \frac{\boldsymbol{w}_{m+1}}{||\boldsymbol{w}_{m+1}||}
    \end{bmatrix}\in\mathbb{R}^{n\times (m+1)}\,,
    \quad 
    \bar{H} :=
    \begin{bmatrix}
    H\\
    ||\boldsymbol{w}_{m+1}|| \boldsymbol{e}_m ^T
    \end{bmatrix}\in\mathbb{R}^{(m+1)\times m}\,.
    $$

## Krylov Methods

After the (very) brief recap about the Krylov subspaces, we make a recap about the Krylov methods used in this laboratory.

### The Arnoldi Method

Given a square matrix $A\in\mathbb{R}^{n\times n}$, a non-null vector $\boldsymbol{v}\in\mathbb{R}^n$, and a fixed $m_0\in\mathbb{N}$, $m_0\leq n$, the *Arnoldi* (A) method is an iterative method that let us compute:

1. The dimension $m$ of $\mathcal{K}_{m_0}(A, \boldsymbol{v})$;

1. The matrix $V\in\mathbb{R}^{n\times m}$ representing the orthogonal basis of $\mathcal{K}_m(A, \boldsymbol{v})$;

1. consequently, the Hessenberg matrix $H=V^T A V$.

Actually, we can observe that the Arnoldi method is just a generalization of the Gram-Schmidt method to the Krylov subspaces.

The **pseudo-code** of the Arnoldi method is the following:

1. $\boldsymbol{v}_1\gets \boldsymbol{v}/||\boldsymbol{v}||$;

1. **for** $j=1,\ldots ,m_0$ **do:** $\quad$*(iteration to build $\boldsymbol{v}_{j+1}$)*
    
    1. **for** $i=1,\ldots ,j$ **do:**
        
        1. $h_{ij} \gets (A\boldsymbol{v}_j)^T \boldsymbol{v_i}$;
        
    1. $\boldsymbol{w}_{j+1}\gets A\boldsymbol{v}_j - \sum_{i=1}^j h_{ij}\boldsymbol{v}_i$;
    
    1. $h_{j+1, j}\gets ||\boldsymbol{w}_{j+1}||$;
    
    1. **if** $h_{j+1,j}=0$ **do:**
    
        1. stop and **return:** $\mathrm{dim}(\mathcal{K}_{m_0}(A,\boldsymbol{v}))=j$, $V=[\boldsymbol{v}_1|\cdots | \boldsymbol{v}_j]$, $H=V^T A V$.
    
    1. **else:**
        
        1. $\boldsymbol{v}_{j+1} = \boldsymbol{w}_{j+1}/h_{j+1, j}$;

1. **return:** $\mathrm{dim}(\mathcal{K}_{m_0}(A,\boldsymbol{v}))$, $V$, $H$, $\boldsymbol{w}_{m+1}$.




#### Exercise 1: Arnoldi Method

Complete the function in the following cell, such that it permorms the Arnoldi method for any square matrix, any vector and any order for the Krylov subspace. The function must return not only $V$ and $H$, but also $\boldsymbol{w}_{m+1}$ (if non-null) and the following norms representing the quality of the outputs:
- $||\mathbb{I}_m - V^{T}V||$;
- $||V^T A V - H||$;
- $||AV - \bar{V}\bar{H}||$, if $\boldsymbol{w}_{m+1}$ is non-null.


In [ ]:
def arnoldi(A, v, m0, tol=1e-8):
    """
    Function that compute an orthogonal basis of an order-m Krylov subspace.
    :param A: square matrix (numpy 2D-array) 
    :param v: nonzero vector (numpy 1D-array or 2D-array)
    :param m0: order of the Krylov subspace w.r.t. A and v
    :param tol: tolerance to stop the algorithm in case the dimension is less than the order
    :return V: n-by-m orth. matrix (column are basis of the Krylov subsp.)
    :return H: m-by-m upper Hessenberg matrix w.r.t. V
    :return m: dimension of the Krylov subspace
    :return Vqual: norm ||Im - V.T @ V||
    :return AVqual1: norm ||V.T @ A @ V - H||
    :return AVqual2: norm ||A @ V - Vbar @ Hbar||
    """
    
    # Parameter initializations
    n = v.size
    
    # Initializations
    # 
    # OBSERVATION: If m=m0 < grade(A,v), we want to compute Vbar and Hbar. Then,
    # we compute directly them and, at the end, we extract V and H

    # Vbar AND Hbar INITIALIZATION
    ...  # <-- TODO!

    for j in range(m0):
        
        ...  # <-- TODO!
        
        # "Dimension Check"
        if abs(Hbar[j + 1, j]) < tol:
            break        
        
        ...  # <-- TODO!
    
    # Compute the dimension
    m = j + 1
    
    # Extract the actual V, H, Vbar, Hbar from "working" Vbar and Hbar matrices  
    ...  # <-- TODO!
    
    Vqual = np.linalg.norm(np.identity(m) - V.T @ V)
    AVqual1 = np.linalg.norm(V.T @ A @ V - H)
    
    if abs(Hbar[j + 1, j]) < tol:
        AVqual2 = None
    else:
        AVqual2 = np.linalg.norm(A @ V - Vbar @ Hbar)

    return V, H, m, Vqual, AVqual1, AVqual2

In [ ]:
# ARNOLDI METHOD TEST

m0_values = range(1, 7)

for m0 in m0_values:
    for ii in range(5):
        print('*')
    print(f'********** Running Arnoldi on A0, v, m={m0}...')
    V, H, m, Vqual, AVqual1, AVqual2 = arnoldi(A0, v, m0)
    print('')
    
    print(f'********** order-{m0} Krylov subspace Dimension: {m}')
    print('')
    
    print('********** Matrix V of A0 **********')
    print(V)
    print('')
    print(f'********** "Orthogonal Quality" of V: {Vqual}')
    print('')

    print('********** Matrix H of A0 **********')
    print(H)
    print('')
    print(f'********** "Factorization Quality" of A0:')
    print(f'AVqual1 = {AVqual1}')
    print(f'AVqual2 = {AVqual2}')
    print('')

### The Modified Arnoldi Method

Since the Arnoldi method is a generalization of the Gram-Schmidt method for the Krylov subspaces, we can define a more stable method: the **Modified Arnoldi** method.

The Modified Arnoldi method, changes the operations in this way:

1. $\boldsymbol{v}_1\gets \boldsymbol{v}/||\boldsymbol{v}||$;

1. **for** $j=1,\ldots ,m_0$ **do:** $\quad$*(iteration to build $\boldsymbol{v}_{j+1}$)*
    1. $\boldsymbol{w}_{j+1}\gets A\boldsymbol{v}_j$;
    
    1. **for** $i=1,\ldots ,j$ **do:**
        
        1. $h_{ij} \gets \boldsymbol{w}_{j+1}^T \boldsymbol{v_i}$;
        
        1. $\boldsymbol{w}_{j+1}\gets \boldsymbol{w}_{j+1} - h_{ij}\boldsymbol{v}_i$;
        
    
    1. $h_{j+1, j}\gets ||\boldsymbol{w}_{j+1}||$;
    
    1. *... equal to the "standard" method...*


#### Exercise 2: Modified Arnoldi Method

Complete the function in the following cell, such that it performs the Modified Arnoldi method for any square matrix.

In [ ]:
def mod_arnoldi(A, v, m0, tol=1e-8):
    """
    Function that compute an orthogonal basis of an order-m Krylov subspace.
    :param A: square matrix (numpy 2D-array) 
    :param v: nonzero vector (numpy 1D-array or 2D-array)
    :param m0: order of the Krylov subspace w.r.t. A and v
    :param tol: tolerance to stop the algorithm in case the dimension is less than the order
    :return V: n-by-m orth. matrix (column are basis of the Krylov subsp.)
    :return H: m-by-m upper Hessenberg matrix w.r.t. V
    :return m: dimension of the Krylov subspace
    :return Vqual: norm ||Im - V.T @ V||
    :return AVqual1: norm ||V.T @ A @ V - H||
    :return AVqual2: norm ||A @ V - Vbar @ Hbar||
    """
    
    ...  # <-- TODO!

    return V, H, m, Vqual, AVqual1, AVqual2

In [ ]:
# ARNOLDI METHOD TEST

m0_values = range(1, 7)

for m0 in m0_values:
    for ii in range(5):
        print('*')
    print(f'********** Running Arnoldi on A0, v, m={m0}...')
    V, H, m, Vqual, AVqual1, AVqual2 = mod_arnoldi(A0, v, m0)
    print('')
    
    print(f'********** order-{m0} Krylov subspace Dimension: {m}')
    print('')
    
    print('********** Matrix V of A0 **********')
    print(V)
    print('')
    print(f'********** "Orthogonal Quality" of V: {Vqual}')
    print('')

    print('********** Matrix H of A0 **********')
    print(H)
    print('')
    print(f'********** "Factorization Quality" of A0:')
    print(f'AVqual1 = {AVqual1}')
    print(f'AVqual2 = {AVqual2}')
    print('')

### The (Basic) Full Orthogonalization Method



Given a square linear system $A\boldsymbol{x} = \boldsymbol{b}$, a "starting-guess" vector $\boldsymbol{x}_0$, and an order $m_0\leq n$, the method basic *Full Orthogonalization Method* (FOM) computes the (approximated) solution $\boldsymbol{x}_{m}$ of the system using the matrices $V$ and $H$ returned by the
Modified Arnoldi method applied to $A$ and $\boldsymbol{r}_0 = \boldsymbol{b} - A\boldsymbol{x}_0$. 

In particular, the FOM exploits the property 
$$\boldsymbol{x}^*=\boldsymbol{x}_0 + \boldsymbol{v}\,,$$
where $\boldsymbol{v}\in\mathcal{K}_{n}(A,\boldsymbol{r}_0)$ is the unknow result of the product $A^{-1}\boldsymbol{r}_0$, and imposes the *Galerkin Orthogonality Condition* (GOC)
for the residual $\boldsymbol{r}_m = \boldsymbol{b} - A \boldsymbol{x}_m$ with respect to the Krylov subspace $\mathcal{K}_{m_0}(A, \boldsymbol{r}_0)$. In a nutshell, we compute a solution $\boldsymbol{x}_m = \boldsymbol{x}_0 + \boldsymbol{v}\approx \boldsymbol{x}^*$, finding a $\boldsymbol{v}\in\mathcal{K}_{m_0}(A,\boldsymbol{r}_0)$ such that $\boldsymbol{r}_m \perp \mathcal{K}_{m_0}(A,\boldsymbol{r}_0)$.

Let us re-write the unknown vector $\boldsymbol{v}$ as $V\boldsymbol{y}$, with $\boldsymbol{y}\in\mathbb{R}^m$ unknown. Then, imposing the GOC above, actually we impose that $V^T\boldsymbol{r}_m=\boldsymbol{0}\in\mathbb{R}^n$ and, therefore, it holds that 
$$
\begin{cases}
H\boldsymbol{y}= ||\boldsymbol{r}_0||\boldsymbol{e}_1\\
\boldsymbol{x}_m = \boldsymbol{x}_0 + V\boldsymbol{y}
\end{cases}\,,
$$
because
\begin{equation*}
\begin{aligned}
\boldsymbol{0} &= V^T\boldsymbol{r}_m = \\
& = V^T(\boldsymbol{b}-A\boldsymbol{x}_m) = V^T(\boldsymbol{b}-A\boldsymbol{x}_0 - AV\boldsymbol{y})=\\
&=V^T(\boldsymbol{r}_0 - AV\boldsymbol{y}) = V^T\boldsymbol{r}_0 - V^T AV\boldsymbol{y} = \\
& = ||\boldsymbol{r}_0||\boldsymbol{e}_1 - H\boldsymbol{y}\,,
\end{aligned}
\end{equation*}
where $V^T\boldsymbol{r}_0 = ||\boldsymbol{r}_0 || \boldsymbol{e}_1$ because the columns of $V$ are an orthogonal basis and we can assume that the first column of $V$ is $\boldsymbol{v}_1=\boldsymbol{r}_0/||\boldsymbol{r}_0||$. 

Then, the **pseudocode of the FOM** is:
1. Compute $V$ and $H$ w.r.t. $\mathcal{K}_{m_0}(A,\boldsymbol{r}_0)$ with Mod. Arnoldi;
1. Find $\boldsymbol{y}$ solving the linear system $H\boldsymbol{y}=||\boldsymbol{r}_0||\boldsymbol{e}_1$;
1. Compute $\boldsymbol{x}_m=\boldsymbol{x}_0 + V\boldsymbol{y}$.

#### Exercise 3: Basic FOM

Complete the function in the following cell, such that it permorms the basic FOM for any nonsingular square linear system. The function must return not only the solution $\boldsymbol{x}_m$ and its residual $\boldsymbol{r}_m$, but also the norm of the product $V^T\boldsymbol{r}_m$ (to check the orthogonality of the residual w.r.t. the Krylov subspace).

**Suggestion:** for simplicity, use the numpy solver function *numpy.linalg.solve* to find $\boldsymbol{y}$.

In [ ]:
def fom_basic(A, b, x0, m0, tol=1e-8):
    """
    Function that compute an approximation xm of a linear system solution, 
    using the FOM method (based on mod_arnoldi)
    :param A: invertible matrix (numpy 2D-array)
    :param b: column vector (numpy 2D-array)
    :param x0: column vector, initial guess (numpy 2D-array)
    :param m0: order of the Krylov subspace for the FOM method
    :param tol: tolerance for the mod_arnoldi function
    :return xm: approximation of the solution
    :return rm: residual of the approximated solution
    :Perpqual: norm of (V^t rm) that it should be zero if the residual is
         actually perpendicular to the Krylov subspace.
    """
    ...  # <-- TODO!
    
    return xm, rm, Perpqual

In [ ]:
# ARNOLDI METHOD TEST

x0 = np.zeros_like(b0)
xstar = np.ones_like(b0)  # real solution of the linear system A0x=b0

m0_values = range(1, 7)

print('********** FOM info **********')
print('Matrix A0:')
print(A0)
print('')
print('Vector b0:')
print(b0)
print('')
print('Starting Guess x0:')
print(x0)
print('')
print('Exact Solution xstar:')
print(xstar)
print('')

for m0 in m0_values:
    for ii in range(5):
        print('*')
    print(f'********** Running FOM on A0, x0, m={m0}...')
    xm, rm, Perpqual = fom_basic(A0, b0, x0, m0)
    print('')
    
    print('********** Solution xm of A0x=b0 **********')
    print(xm)
    print('')
    print('********** Residual rm of xm **********')
    print(rm)
    print('')
    print(f'********** "Perpendicularity Quality" of rm: {Perpqual}')
    print('')